# Baseline - Anomaly Analysis run-fhnw-full-1

In [ ]:
experiment_name = "run-fhnw-full-baseline"
model_config_file = "/home/marius/sdo-cli/config/threshold/run-fhnw-full-1-predict.yaml"

In [ ]:
!cat $model_config_file

In [ ]:
#!sdo-cli sood threshold predict --config-file="/home/marius/sdo-cli/config/threshold/run-fhnw-full-1-predict.yaml"

In [ ]:
sample_pred_path = "/home/marius/sdo-cli/output/threshold/predictions/20220812-095307_cevae/predictions.txt"

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import pandas as pd
import os

%config InlineBackend.figure_format = 'retina'

In [ ]:
df = pd.read_csv(sample_pred_path)
df.head()

In [ ]:
df = df.sort_values(by=['score'], ascending=False)
df["score_norm"] = (df["score"]-df["score"].min())/(df["score"].max()-df["score"].min())
df['score_e'] = np.exp(df['score'])
df.head(10)

In [ ]:
df["score"].hist(bins=50, figsize=(14, 9))

In [ ]:
folder_time_format = "%Y%m%d-%H%M%S"
df["t_obs"] = pd.to_datetime(df["t_obs"])
#somehow some images in 2019 are duplicates?
df = df.drop_duplicates(subset=['t_obs'], keep='first')
df = df.set_index('t_obs', drop=False, verify_integrity=True)

In [ ]:
df.describe()

In [ ]:
df.quantile(.95)

In [ ]:
df.head(10)

In [ ]:
df["t_obs"].min()

In [ ]:
df["t_obs"].max()

In [ ]:
df_1h = df.resample("1h").max()

In [ ]:
df_1h.index.name = "timestamp"

In [ ]:
import dask.dataframe as dd

ddf = dd.read_parquet("/mnt/nas05/astrodata01/astroml_data/goes/goes_ts.parquet", engine="pyarrow", calculate_divisions=True)
ddf

In [ ]:
ddf = ddf[(ddf["quality_xrsb"] == 0) & (ddf["quality_xrsa"] == 0)]

In [ ]:
goes_ts = ddf["2011-01-01 00:00":"2020-12-30 23:59:59"].resample("1h").max().compute()
goes_ts.describe()

In [ ]:
goes_ts["xrsb"]

In [ ]:
def plot_goes_and_anomaly_score(df, anomaly_df):
    axes = df.plot(title=f"GOES X-Ray Flux and Normalized Anomaly Score ({experiment_name})", xlabel="Timestamp", ylabel="Watts $m^{-2}$", logy=True, ylim=(1e-9, 1e-2), figsize=(15, 7.5))
    ax2 = axes.twinx()
    ax2.set_yscale("log")
    ax2.set_ylim(1e-9, 1e-2)
    ax2.set_yticklabels([])
    axes.yaxis.grid(True, "major")
    axes.xaxis.grid(False, "major")
    
    ax3 = anomaly_df[["score_norm"]].plot(ax=axes, logy=False, secondary_y=True, color="r")
    ax3.set_ylim(0, 1)
    #ax3.set_ylabel("Normalized Anomaly Score")
    
    h1, l1 = axes.get_legend_handles_labels()
    plt.legend(h1, l1, loc=2)
    
    return axes

In [ ]:
def filter_test_years(df):
    mask = (((df.index >= "2011-01-01 00:00") & (df.index <= "2011-12-30 23:59:59"))
        | ((df.index >= "2014-01-01 00:00") & (df.index <= "2014-12-30 23:59:59"))
        | ((df.index >= "2016-01-01 00:00") & (df.index <= "2016-12-30 23:59:59"))
        | ((df.index >= "2019-01-01 00:00") & (df.index <= "2019-12-30 23:59:59")))
    
    return df.loc[mask]

def only_2011(df):
    mask = (((df.index >= "2011-01-01 00:00") & (df.index <= "2011-12-30 23:59:59")))
    
    return df.loc[mask]

def only_2014(df):
    mask = (((df.index >= "2014-01-01 00:00") & (df.index <= "2014-12-30 23:59:59")))
    
    return df.loc[mask]

def only_2016(df):
    mask = (((df.index >= "2016-01-01 00:00") & (df.index <= "2016-12-30 23:59:59")))
    
    return df.loc[mask]


def only_2019(df):
    mask = (((df.index >= "2019-01-01 00:00") & (df.index <= "2019-12-30 23:59:59")))
    
    return df.loc[mask]
    

In [ ]:
plot_goes_and_anomaly_score(only_2011(goes_ts)["xrsb"], only_2011(df_1h))

In [ ]:
plot_goes_and_anomaly_score(only_2014(goes_ts)["xrsb"], only_2014(df_1h))

In [ ]:
plot_goes_and_anomaly_score(only_2016(goes_ts)["xrsb"], only_2016(df_1h))

In [ ]:
plot_goes_and_anomaly_score(only_2019(goes_ts)["xrsb"], only_2019(df_1h))

In [ ]:
plot_goes_and_anomaly_score(goes_ts["xrsb"], df_1h)

In [ ]:
df_1h.index = df_1h.index.tz_convert(None)

In [ ]:
df_1h.head()

In [ ]:
goes_ts.head()

In [ ]:
stats_df = pd.read_csv("/home/marius/sdo-cli/notebooks/pixel_stats_171A_1d.csv")
stats_df["timestamp"] =  pd.to_datetime(stats_df["timestamp"], utc=True)
stats_df["timestamp"] = stats_df["timestamp"].apply(
            lambda x: x.replace(microsecond=0))
stats_df = stats_df.set_index('timestamp', drop=True)
stats_df

In [ ]:
## Comparing Anomaly Scores to Pixel intensities

def plot_scores_and_pixel_intensities(pixel_df, anomaly_df):
    axes = anomaly_df["score_norm"].plot(title=f"Mean Pixel Intensity AIA 171Å and Normalized Anomaly Score  ({experiment_name})", xlabel="Timestamp", ylabel="Anomaly Score", color="r", figsize=(15, 7.5))
    ax2 = axes.twinx()
    ax2.set_yticklabels([])
    axes.yaxis.grid(True, "major")
    axes.xaxis.grid(False, "major")
    
    ax3 = pixel_df.plot(ax=axes, secondary_y=True, color="tab:blue")
    ax3.set_ylabel("Mean Pixel Intensity")
    
    h1, l1 = axes.get_legend_handles_labels()
    plt.legend(h1, l1, loc=2)
    
    return axes


In [ ]:
plot_scores_and_pixel_intensities(stats_df[["mean_pixel"]], df_1h)

In [ ]:
# TODO
# try to compute the correlation
# look at just one year to really see the correlation
# find the parts where the two values do not match

## Highest scores

In [ ]:
top_obs_times = []

for index, row in df[:1000].iterrows():
    t_obs = row["t_obs"] # .isoformat(timespec='milliseconds').replace("+00:00", "Z") #.replace(microsecond=0)
    top_obs_times.append(t_obs)
    
top_obs_times[:10]

In [ ]:
def spaced_obs_times(df, min_diff_seconds=24*60*60, min_size = 100):
    obs_times = []

    for index, row in df.iterrows():
        t_obs = row["t_obs"]
        has_close_neighbour = False
        for obs_time in obs_times:
            diff = abs((t_obs - obs_time).total_seconds())
            if diff < min_diff_seconds:
                has_close_neighbour = True
                #print(f"ignoring {t_obs} for diff {diff}")
                break
                
            
        if not has_close_neighbour:
            score = row["score_norm"]
            #print(f"found obs time {t_obs} with score {score}")
            obs_times.append(t_obs)
            
        if len(obs_times) >= min_size:
            break

    return obs_times

In [ ]:
spaced_top_obs_times_1d = spaced_obs_times(df)
spaced_top_obs_times_7d = spaced_obs_times(df, min_diff_seconds=24*60*60*7)

In [ ]:
from pathlib import Path
import numpy as np
import os 
import sunpy
from sunpy.visualization.colormaps import cm
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import torchvision
import torch
import torchvision.transforms.functional as F

#inspect an image

#Channels that correspond to HMI Magnetograms 
HMI_WL = ['Bx','By','Bz']
#A colormap for visualizing HMI
HMI_CM = LinearSegmentedColormap.from_list("bwrblack", ["#0000ff","#000000","#ff0000"])

def channel_to_map(name):
    """Given channel name, return colormap"""
    return HMI_CM if name in HMI_WL else cm.cmlist.get('sdoaia%d' % int(name))

def vis(X, cm):
    """Given image, colormap, and visualize results"""
    Xcv = cm(X)
    return (Xcv[:,:,:3]*255).astype(np.uint8)

In [ ]:
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

In [ ]:
from sdo.sood.data.sdo_ml_v2_dataset import get_default_transforms, SDOMLv2NumpyDataset
from torch.utils.data import DataLoader


def get_data_loader(obs_times):
    storage_root = "/home/marius/data/sdomlv2_full/sdomlv2.zarr"
    storage_driver = "fs"
    cache_max_size = 1*1024*1024*2014
    test_year = ["2011", "2014", "2016", "2019"]
    channel= "171A"
    target_size = 256
    mask_limb = False
    mask_limb_radius_scale_factor = 1.0
    transforms = get_default_transforms(
                target_size=target_size, channel=channel, mask_limb=mask_limb, radius_scale_factor=mask_limb_radius_scale_factor)
    dataset = SDOMLv2NumpyDataset(
                storage_root=storage_root,
                storage_driver=storage_driver,
                cache_max_size=cache_max_size,
                year=test_year,
                start=None,
                end=None,
                freq=None,
                obs_times=obs_times[:10],
                irradiance=None,
                irradiance_channel=None,
                goes_cache_dir=None,
                channel=channel,
                transforms=transforms,
                reduce_memory=True
            )
    
    print(f"found dataset with size {len(dataset)}")
    loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=False,
                          drop_last=False,
                          prefetch_factor=2)
    return loader

In [ ]:
from sunpy.visualization.colormaps import cm
from torchvision.utils import make_grid

def show_grid(imgs, ordered_dates, df, ncols=4, channel="171"):
    nrows=int(len(imgs)/ncols)
    if nrows <= 0:
        nrows = 1
        ncols = len(imgs)
    fix, axs = plt.subplots(figsize=(20,9), ncols=ncols, nrows=nrows, squeeze=True)
    row_index = 0
    i = 0
    for t_obs in ordered_dates[:10]:
        t_obs = t_obs.isoformat(timespec='milliseconds').replace("0+00:00", "Z")
        img = imgs[t_obs]
        row = df.loc[t_obs]
        col = i % ncols
        if i != 0 and i % ncols == 0:
            row_index = row_index + 1
        axs[row_index, col].imshow(img)
        img_name = f"{t_obs} {channel}A"
        score = row["score_norm"]
        axs[row_index, col].set_title(f"{img_name}\n with score " + "%.5f" % score)
        axs[row_index, col].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])
        i += 1
    plt.show()

def visualize_batch(loader, ordered_dates, img_df):
    for batch_idx, samples in enumerate(loader):
        X, y = samples
        V = {}
        for x, t_obs in zip(X, y["T_OBS"]):
            x = x.permute(1,2,0) # torch to pillow
            x = np.squeeze(x.numpy())
            v = vis(x, channel_to_map(171))
            V[t_obs] = Image.fromarray(v)
        
        show_grid(V, ordered_dates, df, ncols=5)
        break   
        
        
def visualize_batch_norm(loader, ordered_times, df):
    for batch_idx, samples in enumerate(loader):
        X, y = samples
        V = {}
        for x, t_obs in zip(X, y["T_OBS"]):
            grid = make_grid(x, normalize=True, value_range=(-1.0, 1.0))
            ndarr = grid.mul(255).add_(0.5).clamp_(0, 255).permute(
                    1, 2, 0).to("cpu", torch.uint8).numpy()
            m = cm.cmlist.get('sdoaia%d' % int(171))
            v = np.squeeze(ndarr[:, :, 0])
            v = m(v)
            v = (v[:, :, :3]*255).astype(np.uint8)
            V[t_obs] = Image.fromarray(v)
        show_grid(V, ordered_times, df, ncols=5)
        break  
        
def anomaly_threshold(loader, ordered_times, df):
    for batch_idx, samples in enumerate(loader):
        X, y = samples
        V = {}
        for x, t_obs in zip(X, y["T_OBS"]):
            grid = make_grid(x, normalize=True)
            ndarr = grid.mul(255).add_(0.5).clamp_(0, 255).permute(
                        1, 2, 0).to("cpu", torch.uint8).numpy()
            lower = ndarr.mean() - 2 * ndarr.std()
            upper = ndarr.mean() + 2 * ndarr.std()
            print(lower, upper)
            
            ndarr[ndarr < upper] = 0
            #ndarr[ndarr >= upper] = 255
            
            ndarr = np.invert(ndarr)
            m = cm.cmlist.get('sdoaia%d' % int(171))
            v = np.squeeze(ndarr[:, :, 0])
            v = m(v)
            v = (v[:, :, :3]*255).astype(np.uint8)
            V[t_obs] = Image.fromarray(ndarr)
        show_grid(V, ordered_times, df, ncols=5)
        break

In [ ]:
top_loader = get_data_loader(top_obs_times)

In [ ]:
visualize_batch(top_loader, top_obs_times, df)

In [ ]:
visualize_batch_norm(top_loader, top_obs_times, df)

In [ ]:
anomaly_threshold(top_loader, top_obs_times, df)

### spaced 1 day

In [ ]:
spaced_1d_top_loader = get_data_loader(spaced_top_obs_times_1d)

In [ ]:
visualize_batch(spaced_1d_top_loader, spaced_top_obs_times_1d, df)

In [ ]:
visualize_batch_norm(spaced_1d_top_loader, spaced_top_obs_times_1d, df)

In [ ]:
anomaly_threshold(spaced_1d_top_loader, spaced_top_obs_times_1d, df)

### spaced 7 days

In [ ]:
spaced_7d_top_loader = get_data_loader(spaced_top_obs_times_7d)

In [ ]:
visualize_batch(spaced_7d_top_loader, spaced_top_obs_times_7d, df)

In [ ]:
visualize_batch_norm(spaced_7d_top_loader, spaced_top_obs_times_7d, df)

In [ ]:
anomaly_threshold(spaced_7d_top_loader, spaced_top_obs_times_7d, df)

## Lowest Scores

In [ ]:
df_asc = df.sort_values(by=['score'], ascending=True)
df_asc.head(10)

In [ ]:
bottom_obs_times = []

for index, row in df_asc.head(100).iterrows():
    t_obs = row["t_obs"]
    bottom_obs_times.append(t_obs)
    
bottom_obs_times[:10]

In [ ]:
spaced_bottom_obs_times_1d = spaced_obs_times(df_asc)
spaced_bottom_obs_times_7d = spaced_obs_times(df_asc, min_diff_seconds=24*60*60*7)

In [ ]:
bottom_loader = get_data_loader(bottom_obs_times)

In [ ]:
visualize_batch(bottom_loader, bottom_obs_times, df_asc)

In [ ]:
visualize_batch_norm(bottom_loader, bottom_obs_times, df_asc)

In [ ]:
anomaly_threshold(bottom_loader, bottom_obs_times, df_asc)

### spaced 1 day

In [ ]:
spaced_1d_bottom_loader = get_data_loader(spaced_bottom_obs_times_1d)

In [ ]:
visualize_batch(spaced_1d_bottom_loader, spaced_bottom_obs_times_1d, df)

In [ ]:
visualize_batch_norm(spaced_1d_bottom_loader, spaced_bottom_obs_times_1d, df)

In [ ]:
anomaly_threshold(spaced_1d_bottom_loader, spaced_bottom_obs_times_1d, df_asc)

### spaced 7 days

In [ ]:
spaced_7d_bottom_loader = get_data_loader(spaced_bottom_obs_times_7d)

In [ ]:
visualize_batch(spaced_7d_bottom_loader, spaced_bottom_obs_times_7d, df)

In [ ]:
visualize_batch_norm(spaced_7d_bottom_loader, spaced_bottom_obs_times_7d, df)

In [ ]:
anomaly_threshold(spaced_7d_bottom_loader, spaced_bottom_obs_times_7d, df_asc)